Executive Summary
- Keys & Duplicates: No duplicate (symbol_id, date_id, time_id)
- Date Continuity: Every symbol shows gaps even beyond date_id=700
- Missingness: Missing values appear only in feature columns, more missing values around before date_id=700
- Low_cardinality static features: feature_09/10/11 are time_invariant (or nearly so) within a symbol, but vary across symbols.
- Responders: all nine responders are clipped to [-5, 5]. Means are near 0, but standard deviations differ substantially (~0.38-1.05). Not uniformly standardized.

There are more detailed EDA explorations on Kaggle; here I highlight the key points I used.

# Set Up

In [1]:
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from pipeline.io import cfg, fs, storage_options

pl.Config.set_tbl_rows(10_000)      
pl.Config.set_tbl_cols(10_000)       
pl.Config.set_tbl_width_chars(10_000) 
pl.Config.set_fmt_str_lengths(10_000) 


pd.set_option("display.max_rows", None)     
pd.set_option("display.max_columns", None)   
pd.set_option("display.width", None)         
pd.set_option("display.max_colwidth", None) 

# Raw Data

In [2]:
azure_root = cfg["azure"]["root"]
local_root = cfg["local"]["root"]

# Build raw-partitions listing (ensure every path has az:// scheme)
raw_dir = f"az://{azure_root}/{cfg['paths']['raw_partitions']['rel']}"
raw_matches = fs.glob(f"{raw_dir}/partition_id=*/*.parquet")
raw_az_paths = [f"az://{p}" for p in raw_matches]
raw_data_paths = sorted(raw_az_paths)

lf = pl.scan_parquet(raw_data_paths, storage_options=storage_options)

In [3]:
# columns
lf_columns = lf.collect_schema().names()
lf_columns

['date_id',
 'time_id',
 'symbol_id',
 'weight',
 'feature_00',
 'feature_01',
 'feature_02',
 'feature_03',
 'feature_04',
 'feature_05',
 'feature_06',
 'feature_07',
 'feature_08',
 'feature_09',
 'feature_10',
 'feature_11',
 'feature_12',
 'feature_13',
 'feature_14',
 'feature_15',
 'feature_16',
 'feature_17',
 'feature_18',
 'feature_19',
 'feature_20',
 'feature_21',
 'feature_22',
 'feature_23',
 'feature_24',
 'feature_25',
 'feature_26',
 'feature_27',
 'feature_28',
 'feature_29',
 'feature_30',
 'feature_31',
 'feature_32',
 'feature_33',
 'feature_34',
 'feature_35',
 'feature_36',
 'feature_37',
 'feature_38',
 'feature_39',
 'feature_40',
 'feature_41',
 'feature_42',
 'feature_43',
 'feature_44',
 'feature_45',
 'feature_46',
 'feature_47',
 'feature_48',
 'feature_49',
 'feature_50',
 'feature_51',
 'feature_52',
 'feature_53',
 'feature_54',
 'feature_55',
 'feature_56',
 'feature_57',
 'feature_58',
 'feature_59',
 'feature_60',
 'feature_61',
 'feature_62',
 'feat

## Key uniqueness

In [4]:
keys = ["symbol_id", "date_id", "time_id"]
dup = lf.group_by(keys).agg(
    pl.len().alias("n")
).filter(pl.col("n") > 1).sort("n", descending=True)

dup.collect()


symbol_id,date_id,time_id,n
i8,i16,i16,u32


Findings: there are no duplicates.

## Time Continuity

In [5]:
unique_sd = lf.select(["symbol_id", "date_id"]).unique().sort(["symbol_id", "date_id"])

sd = unique_sd.with_columns([
    pl.col("date_id").diff().over("symbol_id").cast(pl.Int16).alias("date_id_diff"),
]
)
gaps_summary = (
    sd.filter(pl.col("date_id_diff") > 1)
    .group_by("symbol_id")
    .agg([
        pl.len().alias("n_gaps"),
        (pl.col("date_id_diff").sum() - 1).alias("total_missing_dates"),
    ])
    
)

gaps_summary_df = gaps_summary.collect()

gaps_summary_df.sort(["total_missing_dates", "n_gaps"], descending=True)

symbol_id,n_gaps,total_missing_dates
i8,u32,i64
15,60,288
21,21,241
8,27,198
3,43,174
14,38,160
30,20,157
34,39,145
11,35,142
17,40,140


In [6]:
lf_tail = lf.filter(pl.col("date_id") >=700)

unique_sd = lf_tail.select(["symbol_id", "date_id"]).unique().sort(["symbol_id", "date_id"])

sd = unique_sd.with_columns([
    pl.col("date_id").diff().over("symbol_id").cast(pl.Int16).alias("date_id_diff"),
]
)

gaps_summary = (
    sd.filter(pl.col("date_id_diff") > 1)
    .group_by("symbol_id")
    .agg([
        pl.len().alias("n_gaps"),
        pl.col("date_id_diff").max().alias("max_gap_size"),
        pl.col("date_id_diff").min().alias("min_gap_size"),
        (pl.col("date_id_diff").sum() - 1).alias("total_missing_dates"),
    ])
    
)

gaps_summary_df = gaps_summary.collect()

gaps_summary_df.sort(["total_missing_dates", "n_gaps"], descending=True)

symbol_id,n_gaps,max_gap_size,min_gap_size,total_missing_dates
i8,u32,i16,i16,i64
21,9,200,2,216
27,37,34,2,132
5,13,61,2,111
8,13,49,2,95
18,21,31,2,78
35,25,12,2,68
4,21,12,2,61
3,22,8,2,53
0,14,23,2,53


Findings: all symbols exhibit gaps in date coverage; gaps persist even after date_id=700

## Missingness

In [7]:
# overall missingness
lf.select(
    [pl.col(c).null_count().alias(c) for c in lf_columns]
).collect()

date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,3182052,3182052,3182052,3182052,3182052,0,0,0,300247,0,0,0,0,0,0,1209299,261,201838,226,226,0,8435985,0,0,0,0,8435985,8435985,0,0,0,8435985,478457,478457,0,0,0,849,0,4300649,67856,1093012,4300649,67856,1093012,317163,317163,87,0,0,4254098,13805,1044898,4254098,13805,1044898,226,226,478452,0,0,0,292827,227566,237663,317163,317163,0,0,0,0,0,0,483759,483759,58430,58430,20043,20043,0,0,0,0,0,0,0,0,0


Findings: only features have missing values, responders have no missing values

In [8]:
# missingness distribution per feature
responders_cols = [f"responder_{i}" for i in range(0, 9)]
features_cols = [f"feature_{i:02d}" for i in range(0, 79)]
keys = ["date_id", "time_id", "symbol_id"]

In [9]:
# missingness distribution per feature per day
lf_missingness_per_day = (
    lf.group_by("date_id")
    .agg([pl.col(c).is_null().mean().alias(f"{c}_null_frac") for c in features_cols])
    .sort("date_id")
    .collect()
)

In [10]:
missing_long = (
    lf_missingness_per_day
    .melt(id_vars="date_id", variable_name="feature", value_name="null_frac")
    .with_columns(pl.col("feature").str.replace(r"_null_frac$", ""))
)

heat = (
    missing_long
    .to_pandas()
    .pivot(index="feature", columns="date_id", values="null_frac")
    .sort_index()
)
print(heat.shape)  
print(heat.index[:5], heat.columns[:5])

(79, 1699)
Index(['feature_00', 'feature_01', 'feature_02', 'feature_03', 'feature_04'], dtype='object', name='feature') Index([0, 1, 2, 3, 4], dtype='int16', name='date_id')


/tmp/ipykernel_1270699/3931724802.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  lf_missingness_per_day


In [11]:
import numpy as np, matplotlib.pyplot as plt

A = heat.values                  
n_feat, n_day = A.shape

fig, ax = plt.subplots(figsize=(max(12, n_day/20), max(12, n_feat/1.6)), dpi=150)
im = ax.imshow(A, aspect="auto", origin="upper", interpolation="nearest")
fig.colorbar(im, ax=ax, label="missing fraction")

xstep = max(1, n_day // 30)
ystep = max(1, n_feat // 40)
ax.set_xticks(np.arange(0, n_day, xstep)); ax.set_xticklabels(heat.columns[::xstep], rotation=90, fontsize=7)
ax.set_yticks(np.arange(0, n_feat, ystep)); ax.set_yticklabels(heat.index[::ystep], fontsize=7)

ax.set_xlabel("date_id"); ax.set_ylabel("feature")
plt.tight_layout(); plt.show()


# # Plotly version
# import plotly.express as px
# fig = px.imshow(heat, origin="upper", aspect="auto", color_continuous_scale="Viridis",
#                 labels=dict(color="missing frac"))
# fig.write_html("missing_heatmap.html", include_plotlyjs="cdn")  # using browser to view


In [12]:
# Plotly version
import plotly.express as px
fig = px.imshow(heat, origin="upper", aspect="auto", color_continuous_scale="Viridis",
                labels=dict(color="missing frac"))
fig.write_html("missing_heatmap.html", include_plotlyjs="cdn")  # using browser to view

Findings: 
1. most missing values appear before around date_id = 700
2. there are some features which have stable and small missing fraction over all time, such as feature_15,39,42,50,53

## Statistics

### Features

In [13]:
aggs = []
for c in features_cols:
    aggs += [
        pl.col(c).mean().alias(f"{c}_mean"),
        pl.col(c).std().alias(f"{c}_std"),
        pl.col(c).min().alias(f"{c}_min"),
        pl.col(c).max().alias(f"{c}_max"),
    ]
    
feature_stats = lf.group_by("symbol_id").agg(aggs)

In [14]:
feature_stats_df = feature_stats.collect().to_pandas().sort_values(by="symbol_id").reset_index(drop=True)

In [15]:
feature_stats_df

,symbol_id,feature_00_mean,feature_00_std,feature_00_min,feature_00_max,feature_01_mean,feature_01_std,feature_01_min,feature_01_max,feature_02_mean,feature_02_std,feature_02_min,feature_02_max,feature_03_mean,feature_03_std,feature_03_min,feature_03_max,feature_04_mean,feature_04_std,feature_04_min,feature_04_max,feature_05_mean,feature_05_std,feature_05_min,feature_05_max,feature_06_mean,feature_06_std,feature_06_min,feature_06_max,feature_07_mean,feature_07_std,feature_07_min,feature_07_max,feature_08_mean,feature_08_std,feature_08_min,feature_08_max,feature_09_mean,feature_09_std,feature_09_min,feature_09_max,feature_10_mean,feature_10_std,feature_10_min,feature_10_max,feature_11_mean,feature_11_std,feature_11_min,feature_11_max,feature_12_mean,feature_12_std,feature_12_min,feature_12_max,feature_13_mean,feature_13_std,feature_13_min,feature_13_max,feature_14_mean,feature_14_std,feature_14_min,feature_14_max,feature_15_mean,feature_15_std,feature_15_min,feature_15_max,feature_16_mean,feature_16_std,feature_16_min,feature_16_max,feature_17_mean,feature_17_std,feature_17_min,feature_17_max,feature_18_mean,feature_18_std,feature_18_min,feature_18_max,feature_19_mean,feature_19_std,feature_19_min,feature_19_max,feature_20_mean,feature_20_std,feature_20_min,feature_20_max,feature_21_mean,feature_21_std,feature_21_min,feature_21_max,feature_22_mean,feature_22_std,feature_22_min,feature_22_max,feature_23_mean,feature_23_std,feature_23_min,feature_23_max,feature_24_mean,feature_24_std,feature_24_min,feature_24_max,feature_25_mean,feature_25_std,feature_25_min,feature_25_max,feature_26_mean,feature_26_std,feature_26_min,feature_26_max,feature_27_mean,feature_27_std,feature_27_min,feature_27_max,feature_28_mean,feature_28_std,feature_28_min,feature_28_max,feature_29_mean,feature_29_std,feature_29_min,feature_29_max,feature_30_mean,feature_30_std,feature_30_min,feature_30_max,feature_31_mean,feature_31_std,feature_31_min,feature_31_max,feature_32_mean,feature_32_std,feature_32_min,feature_32_max,feature_33_mean,feature_33_std,feature_33_min,feature_33_max,feature_34_mean,feature_34_std,feature_34_min,feature_34_max,feature_35_mean,feature_35_std,feature_35_min,feature_35_max,feature_36_mean,feature_36_std,feature_36_min,feature_36_max,feature_37_mean,feature_37_std,feature_37_min,feature_37_max,feature_38_mean,feature_38_std,feature_38_min,feature_38_max,feature_39_mean,feature_39_std,feature_39_min,feature_39_max,feature_40_mean,feature_40_std,feature_40_min,feature_40_max,feature_41_mean,feature_41_std,feature_41_min,feature_41_max,feature_42_mean,feature_42_std,feature_42_min,feature_42_max,feature_43_mean,feature_43_std,feature_43_min,feature_43_max,feature_44_mean,feature_44_std,feature_44_min,feature_44_max,feature_45_mean,feature_45_std,feature_45_min,feature_45_max,feature_46_mean,feature_46_std,feature_46_min,feature_46_max,feature_47_mean,feature_47_std,feature_47_min,feature_47_max,feature_48_mean,feature_48_std,feature_48_min,feature_48_max,feature_49_mean,feature_49_std,feature_49_min,feature_49_max,feature_50_mean,feature_50_std,feature_50_min,feature_50_max,feature_51_mean,feature_51_std,feature_51_min,feature_51_max,feature_52_mean,feature_52_std,feature_52_min,feature_52_max,feature_53_mean,feature_53_std,feature_53_min,feature_53_max,feature_54_mean,feature_54_std,feature_54_min,feature_54_max,feature_55_mean,feature_55_std,feature_55_min,feature_55_max,feature_56_mean,feature_56_std,feature_56_min,feature_56_max,feature_57_mean,feature_57_std,feature_57_min,feature_57_max,feature_58_mean,feature_58_std,feature_58_min,feature_58_max,feature_59_mean,feature_59_std,feature_59_min,feature_59_max,feature_60_mean,feature_60_std,feature_60_min,feature_60_max,feature_61_mean,feature_61_std,feature_61_min,feature_61_max,feature_62_mean,feature_62_std,feature_62_min,feature_62_max,feature_63_mean,feature_63_std,feature_63_min,feature_63_max,feature_64_mean,feature_64_std,feature_64_min,feature_64_max,feature_65_mean,feature_

In [16]:
for i in range(39):
    print(f"simbol_id: {i}")
    series = feature_stats_df.iloc[0]
    
    for j in range(79):
        feature_std = f"feature_{j:02d}_std"
        if series[feature_std] < 1e-5:
            print(f"  feature_{j:02d} has low std: {series[feature_std]}")
            

simbol_id: 0
  feature_09 has low std: 0.0
  feature_10 has low std: 0.0
  feature_11 has low std: 0.0
simbol_id: 1
  feature_09 has low std: 0.0
  feature_10 has low std: 0.0
  feature_11 has low std: 0.0
simbol_id: 2
  feature_09 has low std: 0.0
  feature_10 has low std: 0.0
  feature_11 has low std: 0.0
simbol_id: 3
  feature_09 has low std: 0.0
  feature_10 has low std: 0.0
  feature_11 has low std: 0.0
simbol_id: 4
  feature_09 has low std: 0.0
  feature_10 has low std: 0.0
  feature_11 has low std: 0.0
simbol_id: 5
  feature_09 has low std: 0.0
  feature_10 has low std: 0.0
  feature_11 has low std: 0.0
simbol_id: 6
  feature_09 has low std: 0.0
  feature_10 has low std: 0.0
  feature_11 has low std: 0.0
simbol_id: 7
  feature_09 has low std: 0.0
  feature_10 has low std: 0.0
  feature_11 has low std: 0.0
simbol_id: 8
  feature_09 has low std: 0.0
  feature_10 has low std: 0.0
  feature_11 has low std: 0.0
simbol_id: 9
  feature_09 has low std: 0.0
  feature_10 has low std: 0.0


In [17]:
# Check unique values for feature_09, feature_10, feature_11 per group
lf.group_by("symbol_id").agg(
    pl.col("feature_09").drop_nulls().n_unique().alias("feature_09_nunique"),
    pl.col("feature_10").drop_nulls().n_unique().alias("feature_10_nunique"),
    pl.col("feature_11").drop_nulls().n_unique().alias("feature_11_nunique"),
).sort("symbol_id").collect()
    

symbol_id,feature_09_nunique,feature_10_nunique,feature_11_nunique
i8,u32,u32,u32
0,1,1,1
1,1,1,1
2,1,1,1
3,1,1,1
4,1,1,1
5,1,1,1
6,1,1,2
7,1,1,1
8,1,1,1


In [18]:
lf_tail.group_by("symbol_id").agg(
    pl.col("feature_09").drop_nulls().n_unique().alias("feature_09_nunique"),
    pl.col("feature_10").drop_nulls().n_unique().alias("feature_10_nunique"),
    pl.col("feature_11").drop_nulls().n_unique().alias("feature_11_nunique"),
).sort("symbol_id").collect()

symbol_id,feature_09_nunique,feature_10_nunique,feature_11_nunique
i8,u32,u32,u32
0,1,1,1
1,1,1,1
2,1,1,1
3,1,1,1
4,1,1,1
5,1,1,1
6,1,1,2
7,1,1,1
8,1,1,1


In [19]:
# overall unique values for feature_09, feature_10, feature_11
lf.select(
    pl.col("feature_09").unique().count().alias("feature_09_nunique"),
    pl.col("feature_10").unique().count().alias("feature_10_nunique"),
    pl.col("feature_11").unique().count().alias("feature_11_nunique"),
).collect()

feature_09_nunique,feature_10_nunique,feature_11_nunique
u32,u32,u32
22,9,31


In [20]:
lf.select(
    pl.col("feature_09").drop_nulls().n_unique().alias("f09_nunique_no_null"),
    pl.col("feature_10").drop_nulls().n_unique().alias("f10_nunique_no_null"),
    pl.col("feature_11").drop_nulls().n_unique().alias("f11_nunique_no_null"),
).collect()


f09_nunique_no_null,f10_nunique_no_null,f11_nunique_no_null
u32,u32,u32
22,9,31


In [21]:
lf_tail.select(
    pl.col("feature_09").drop_nulls().n_unique().alias("f09_nunique_no_null"),
    pl.col("feature_10").drop_nulls().n_unique().alias("f10_nunique_no_null"),
    pl.col("feature_11").drop_nulls().n_unique().alias("f11_nunique_no_null"),
).collect()

f09_nunique_no_null,f10_nunique_no_null,f11_nunique_no_null
u32,u32,u32
22,9,30


Findings: feature_09/10/11 are time-invariant (or nearly so) within each symbol but vary across symbols. Treat them as low-cardinality, symbol-level categorical features (with time safety) and evaluate their cross-sectional impact on the target

### Responders

In [22]:
aggs = []
for i in range(9):
    responders = f"responder_{i}"
    aggs += [
        pl.col(responders).mean().alias(f"{responders}_mean"),
        pl.col(responders).std().alias(f"{responders}_std"),
        pl.col(responders).min().alias(f"{responders}_min"),
        pl.col(responders).max().alias(f"{responders}_max")
    ]
    
responder_stats = lf.group_by("symbol_id").agg(aggs)
responder_stats_df = responder_stats.collect().to_pandas().sort_values(by="symbol_id").reset_index(drop=True)
responder_stats_df

,symbol_id,responder_0_mean,responder_0_std,responder_0_min,responder_0_max,responder_1_mean,responder_1_std,responder_1_min,responder_1_max,responder_2_mean,responder_2_std,responder_2_min,responder_2_max,responder_3_mean,responder_3_std,responder_3_min,responder_3_max,responder_4_mean,responder_4_std,responder_4_min,responder_4_max,responder_5_mean,responder_5_std,responder_5_min,responder_5_max,responder_6_mean,responder_6_std,responder_6_min,responder_6_max,responder_7_mean,responder_7_std,responder_7_min,responder_7_max,responder_8_mean,responder_8_std,responder_8_min,responder_8_max
0,0,-0.003263,0.448974,-5.0,5.0,-0.004957,0.476175,-5.0,5.0,-0.000454,0.467593,-5.0,5.0,-0.014453,0.682835,-5.0,5.0,-0.012524,0.707468,-5.0,5.0,-0.015017,0.632727,-5.0,5.0,-0.002568,0.722675,-5.0,5.0,0.001957,0.725524,-5.0,5.0,-0.001849,0.742503,-5.0,5.0
1,1,-0.000864,0.389497,-5.0,5.0,0.001913,0.382348,-5.0,5.0,-0.000126,0.420736,-5.0,5.0,-0.009126,0.644241,-5.0,5.0,-0.014843,0.679967,-5.0,5.0,-0.008625,0.594000,-5.0,5.0,-0.003720,0.712762,-5.0,5.0,-0.008598,0.749535,-5.0,5.0,-0.001045,0.707489,-5.0,5.0
2,2,-0.004738,0.526888,-5.0,5.0,-0.021871,0.634510,-5.0,5.0,-0.000528,0.515500,-5.0,5.0,-0.019409,0.823045,-5.0,5.0,-0.014517,0.916240,-5.0,5.0,-0.018480,0.710550,-5.0,5.0,-0.000269,0.895544,-5.0,5.0,0.008310,0.925013,-5.0,5.0,0.000302,0.870388,-5.0,5.0
3,3,0.006736,0.831280,-5.0,5.0,0.024102,0.800951,-5.0,5.0,0.001982,0.826140,-5.0,5.0,-0.013950,0.829397,-5.0,5.0,-0.004413,0.897445,-5.0,5.0,-0.018497,0.738977,-5.0,5.0,0.002626,0.898519,-5.0,5.0,0.005551,0.938366,-5.0,5.0,0.002520,0.862264,-5.0,5.0
4,4,-0.004007,0.498329,-5.0,5.0,-0.009836,0.459348,-5.0,5.0,0.000023,0.531933,-5.0,5.0,-0.033984,0.831207,-5.0,5.0,-0.019851,0.894129,-5.0,5.0,-0.036822,0.742839,-5.0,5.0,0.003154,0.926699,-5.0,5.0,0.020490,0.970888,-5.0,5.0,0.003357,0.891889,-5.0,5.0
5,5,-0.001703,0.387409,-5.0,5.0,0.002691,0.370649,-5.0,5.0,-0.000839,0.396880,-5.0,5.0,-0.019742,0.724826,-5.0,5.0,-0.025394,0.794410,-5.0,5.0,-0.017617,0.634538,-5.0,5.0,-0.004584,0.818448,-5.0,5.0,-0.014820,0.866402,-5.0,5.0,0.000793,0.782531,-5.0,5.0
6,6,-0.003264,0.593543,-5.0,5.0,-0.013612,0.557833,-5.0,5.0,-0.000099,0.647271,-5.0,5.0,-0.038413,0.894313,-5.0,5.0,-0.025399,0.918728,-5.0,5.0,-0.038356,0.804274,-5.0,5.0,0.002364,0.955785,-5.0,5.0,0.016159,0.970989,-5.0,5.0,0.002039,0.912777,-5.0,5.0
7,7,-0.002137,0.644657,-5.0,5.0,0.002815,0.571105,-5.0,5.0,-0.000531,0.673947,-5.0,5.0,-0.010772,0.796377,-5.0,5.0,-0.005073,0.783639,-5.0,5.0,-0.013617,0.763185,-5.0,5.0,-0.001084,0.836084,-5.0,5.0,0.004979,0.811533,-5.0,5.0,-0.002877,0.863635,-5.0,5.0
8,8,-0.000943,0.594158,-5.0,5.0,0.007739,0.546279,-5.0,5.0,0.000182,0.591366,-5.0,5.0,-0.012491,0.849319,-5.0,5.0,-0.015041,0.872199,-5.0,5.0,-0.015049,0.776256,-5.0,5.0,0.001110,0.862126,-5.0,5.0,-0.005584,0.900057,-5.0,5.0,0.002185,0.827055,-5.0,5.0
9,9,0.001309,0.510921,-5.0,5.0,0.014151,0.517380,-5.0,5.0,0.000557,0.551512,-5.0,5.0,-0.019110,0.724423,-5.0,5.0,-0.043801,0.804751,-5.0,5.0,-0.012062,0.653172,-5.0,5.0,-0.024897,0.819305,-5.0,5.0,-0.046990,0.854044,-5.0,5.0,-0.014066,0.806105,-5.0,5.0


Findings: All nine responders are clipped to [-5, 5]. Their means are close to zero, while their standard deviations vary substantially (≈0.38–1.05).